# Wrangle OpenStreetMap Data
## 1. Chosen Area
I'm going to wrangle on the coast of Pernambuco, Brazil openstreetmap data. In this area alone lives roghly 5 million people. It also has some of the most beautiful Brazilian beaches and it is also where I live :).

## 2. Objective 
The objective here is to work with streets only fixing their prefixes. In Portuguese the streets types such as Street, Avenue, etc comes first and not last as in Engligh so that a street called Silva Street in English is Rua Silva in Portuguese.<br>
A secondary objective is to find what and where other issues at this dataset are and give some options on how those issues could be fixed and the dataset quality improved.

## 3. Wrangling
### 3.1. OSM File
The first step was to select the area and download its osm file from the openstreetmap website. I named the file **PE_Coast.osm** and has around 100MB.<br>
-rw-r--r--@  1 leo  staff  98919948 Sep 10 10:35 PE_Coast.osm

In [51]:
OSM_FILE = "PE_Coast.osm"  # Full OSM FIle

### 3.2. Wrangling Process
#### 3.3.1 Tags Counting
Started by counting the tags to get a sense of how many **way** tags we have in the file.<br>
**'way': 66137**

In [54]:
# counting tags to get a sense of how many of each tags we have
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict

def count_tags(filename):
    tags = defaultdict(int)
    for event, elem in ET.iterparse(filename):
        tags[elem.tag] += 1
    
    return tags

tags = count_tags(OSM_FILE)
pprint.pprint(tags)

defaultdict(<type 'int'>, {'node': 430343, 'member': 10359, 'nd': 546941, 'tag': 175116, 'bounds': 1, 'note': 1, 'meta': 1, 'relation': 760, 'way': 66137, 'osm': 1})


#### 3.3.2 Street Types Counts
To get a sense of how many different street types and their counts I'll count and print all different street types below (prefixes).

In [46]:
# In Brazil the street type comes first (Avenue, Street, etc)
# This snippet checks all the street names prefixes in teh OSM file to see if there's anything strange or not expected as the street type
# The definition of street is: it's a way tag with a tag with k equals to highway

def check_street_types_counts():
    street_types_counts = defaultdict(int)
    for _, element in ET.iterparse(OSM_FILE):
        if element.tag == "way":
            is_street = False
            street_name = None
            for tag in element.iter("tag"):
                if tag.get("k") == "highway":
                    is_street = True
                elif tag.get("k") == "name":
                    street_name = tag.get("v")
#                     print street_name
            if is_street and street_name is not None:
                street_type = street_name.split(" ")[0]
#                 print street_type
                street_types_counts[street_type] += 1

    return street_types_counts

street_types_counts = check_street_types_counts()
pprint.pprint(dict(street_types_counts))

{'1': 2,
 '15': 1,
 u'1a': 24,
 u'1\xaa': 35,
 u'1\xb0': 7,
 '2a': 18,
 u'2\xaa': 26,
 u'2\xb0': 3,
 u'3': 1,
 '3a': 10,
 u'3\xaa': 18,
 u'3\xb0': 1,
 '4a': 7,
 u'4\xaa': 8,
 u'4\xb0': 1,
 '5a': 6,
 u'5\xaa': 2,
 u'5\xb0': 4,
 '6a': 2,
 u'6\xaa': 1,
 '7a': 2,
 u'7\xaa': 2,
 u'8\xaa': 1,
 u'Acesso': 60,
 'Alameda': 34,
 'Alto': 3,
 u'Al\xe7a': 1,
 u'Ant\xf4nia': 1,
 'Av': 7,
 'Av.': 9,
 'Avenida': 1479,
 'Base': 1,
 'Beco': 22,
 u'Boa': 1,
 u'Cais': 11,
 u'Caminho': 7,
 u'Canal': 3,
 u'Ciclovia': 2,
 u'Condom\xednio': 17,
 u'Corredor': 14,
 u'C\xf3rrego': 7,
 'Dique': 1,
 u'Engenho': 2,
 'Entrada': 8,
 'Escadaria': 1,
 'Estrada': 225,
 'FCA': 25,
 u'Feira': 1,
 'Fiat': 1,
 u'Girador': 3,
 u'Gon\xe7alves': 1,
 'Ladeira': 5,
 'Largo': 12,
 'Loteamento': 3,
 u'Metr\xf4/': 1,
 u'Metr\xf4/Aeroporto': 1,
 u'Mini': 1,
 'Nossa': 1,
 'PE': 9,
 'PE-001': 1,
 'PE-007': 1,
 'PE-009': 1,
 'PE-022': 26,
 'PE-051': 2,
 'PE-060': 13,
 'PE-061': 4,
 'PE-064': 4,
 'PE-070': 1,
 'PE-073': 5,
 'PE-090': 1,

#### 3.3.3 Street Types Counts Ordered
I'll order the list above by the counts so we can see what are the most important street prefixes to tackle.<br>
To help non-Portugues speakers I'll translate some of the names found below:<br>
Rua - Street<br>
Avenida - Avenue<br>
Estrada - Road<br>
Praça - Park<br>
Ponte - Bridge<br>

In [49]:
# print the street_types_counts dictionary, only sorted by count
sorted_dict = sorted(street_types_counts.items(), key=lambda x:x[1], reverse = True)

pprint.pprint(sorted_dict)

[(u'Rua', 13322),
 ('Avenida', 1479),
 ('Travessa', 431),
 ('Estrada', 225),
 (u'Pra\xe7a', 127),
 ('Rodovia', 97),
 (u'Acesso', 60),
 ('Ponte', 59),
 (u'1\xaa', 35),
 ('Alameda', 34),
 ('PE-022', 26),
 (u'2\xaa', 26),
 ('FCA', 25),
 (u'1a', 24),
 ('Beco', 22),
 ('Vila', 22),
 (u'Via', 21),
 ('Viaduto', 20),
 (u'3\xaa', 18),
 ('2a', 18),
 (u'Primeira', 17),
 (u'Condom\xednio', 17),
 (u'Corredor', 14),
 ('PE-060', 13),
 ('Largo', 12),
 ('Segunda', 12),
 (u'Cais', 11),
 (u'P\xe1tio', 10),
 ('3a', 10),
 ('Ramal', 10),
 ('travessa', 10),
 ('PE', 9),
 ('PE-35', 9),
 ('Av.', 9),
 ('Entrada', 8),
 (u'Sa\xedda', 8),
 (u'4\xaa', 8),
 (u'C\xf3rrego', 7),
 (u'Caminho', 7),
 (u'T\xfanel', 7),
 ('Paralela', 7),
 (u'1\xb0', 7),
 ('Av', 7),
 ('4a', 7),
 ('5a', 6),
 ('Ladeira', 5),
 ('Quadra', 5),
 ('rua', 5),
 ('PE-073', 5),
 (u'5\xb0', 4),
 (u'Quarta', 4),
 ('PE-061', 4),
 ('PE-064', 4),
 (u'Terceira', 4),
 ('Alto', 3),
 ('Loteamento', 3),
 (u'2\xb0', 3),
 (u'Canal', 3),
 (u'Girador', 3),
 ('Shoppin

#### 3.3.5 Getting Way Ids To Manually Check Some Entries
I found some of the prefixes in the list above a bit strange so I decided to check some of them directly in the file. For that I need the respective way ids related to each street name I want to manually check. <br>
Example: for the first item inthe list printed below we have:<br>
'1': set(['141891164', '141891277']<br>
That means that ways **141891164** and **141891277** have street names that start with **1**<br>

In [58]:
# map the street prefixes with the respective way ids
# used to manually check some odd results I found at first glance

def check_street_types_way_id():
    street_types_ways = defaultdict(set)
    for _, element in ET.iterparse(OSM_FILE):
        if element.tag == "way":
            way_id = element.get("id")
            is_street = False
            street_name = None
            for tag in element.iter("tag"):
                if tag.get("k") == "highway":
                    is_street = True
                elif tag.get("k") == "name":
                    street_name = tag.get("v")
            if is_street:
                if None == street_name:
                    street_types_ways["None"].add(way_id)
                else:
                    street_type = street_name.split(" ")[0]
                    street_types_ways[street_type].add(way_id)
    return street_types_ways

street_types_ways = check_street_types_way_id()
pprint.pprint(dict(street_types_ways))

{'1': set(['141891164', '141891277']),
 '15': set(['128145838']),
 u'1a': set(['240888784',
             '242642168',
             '249110647',
             '249471288',
             '250183714',
             '250734159',
             '250734160',
             '250738439',
             '250813232',
             '250819849',
             '250819859',
             '251074191',
             '251076695',
             '251393992',
             '251404982',
             '251737135',
             '251903574',
             '252671277',
             '252671280',
             '252970410',
             '254490506',
             '370764358',
             '370955520',
             '371357556']),
 u'1\xaa': set(['100351206',
                '140588114',
                '141026888',
                '141867078',
                '141867261',
                '142217564',
                '151112444',
                '153887409',
                '161395002',
                '161797655',
                '1

#### 3.3.6 Creating the Replacement Dictionary
After manually checking some of the entries that I found odd using the dictionary above I now can create the dicttionary below that simple manps the bad or wrong street name prefixes found in the file with the right ones

In [59]:
# replacement dictionary to be used to clean up the street prefixes
# this dictionary had been built by analyzing the information from the data mungling steps above
replacement_map = {"travessa": "Travessa",
"Travesa": "Travessa",
"Av.": "Avenida",
"Av": "Avenida",
"R.": "Rua",
"rua": "Rua",
"Ria": "Rua",
"rUA": "Rua",
"segunda": "2a",
"Terceira": "3a",
"Quarta": "4a",
"1°": "1a",
"2°": "2a",
"3°": "3a",
"4°": "4a",
"5°": "5a",
"6°": "6a",
"7°": "7a",
"8°": "8a",
"1ª": "1a",
"2ª": "2a",
"3ª": "3a",
"4ª": "4a",
"5ª": "5a",
"6ª": "6a",
"7ª": "7a",
"8ª": "8a",
"1 Travessa": "1a Travessa",
"2 Travessa": "2a Travessa",
"3 Travessa": "3a Travessa",
"4 Travessa": "4a Travessa",
"5 Travessa": "5a Travessa",
"6 Travessa": "6a Travessa",
"7 Travessa": "7a Travessa",
"8 Travessa": "8a Travessa",
"Primeira": "1a Travessa",
"Segunda": "2a Travessa",
"Terceira": "3a Travessa",
"Quarta": "4a Travessa"}

#### 3.3.7 Creating the Replacement Dictionary
Now it's time to use the dictionary to replace the bad prefixes. As I'm already looping through the entire file to read and fix the street prefixes I will use this opportunity to create the json file to be imported into mongdb int the next step.

Note that I'm getting as much information as I find valuable to make further analysis such as the tags lit, surface, maxspeed and many others.

In [61]:
import codecs
import json

# replaces the bad string prefixes (street types)
def replace_street_prefix(name):
    for key in replacement_map:
#         print key
#         print name.split(" ")[0]
#         print name.split(" ")[0] == key
        if name.split(" ")[0] == key:
            name.replace(key, replacement_map[key], 1)
            return name.replace(key, replacement_map[key], 1)
        else:
            return name

# processes each element from the open street map xml file
# returns a dictionary of this element
def process_element(element):
    way = {}
    if element.tag == "way":
        is_street = False
        way["name"] = None
        way_element = element
        for tag in element.iter("tag"):
            if tag.get("k") == "highway":
                is_street = True
                way["highway"] = tag.get("v")
            elif tag.get("k") == "name":
                way["name"] = tag.get("v")
            elif tag.get("k") == "oneway":
                way["oneway"] = tag.get("v")
            elif tag.get("k") == "surface":
                way["surface"] = tag.get("v")
            elif tag.get("k") == "lit":
                way["lit"] = tag.get("v")
            elif tag.get("k") == "access":
                way["access"] = tag.get("v")
            elif tag.get("k") == "emergency":
                way["emergency"] = tag.get("v")
            elif tag.get("k") == "lanes":
                way["lanes"] = int(tag.get("v"))
            elif tag.get("k") == "layer":
                way["layer"] = int(tag.get("v"))
            elif tag.get("k") == "maxspeed":
                way["maxspeed"] = int(tag.get("v"))
            elif tag.get("k") == "bridge":
                way["bridge"] = tag.get("v")
            elif tag.get("k") == "noexit":
                way["noexit"] = tag.get("v")
            elif tag.get("k") == "cycleway" or tag.get("k") == "cycleway:right" or tag.get("k") == "cycleway:left":
                way["cycleway"] = tag.get("v")
            elif tag.get("k") == "sidewalk":
                way["sidewalk"] = tag.get("v")
            elif tag.get("k") == "amenity":
                way["amenity"] = tag.get("v")

        if is_street:
#             pprint.pprint(way["name"])
            if way["name"] is not None:
                way["name"] = replace_street_prefix(way["name"])
            way["id"] = way_element.get("id")
            way["uid"] = way_element.get("uid")
            way["user"] = way_element.get("user")
            way["timestamp"] = way_element.get("timestamp")
            way["version"] = int(way_element.get("version"))
#             pprint.pprint(way["name"])
            return way 
        
    return None

# generates the json file to be imported into mongodb using the mongoimport command
def process_file(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}_processed.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = process_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

process_file(OSM_FILE, True)

/Users/leo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


[{'highway': 'residential',
  'id': '23389996',
  'lit': 'yes',
  'name': None,
  'timestamp': '2016-09-03T17:57:02Z',
  'uid': '367813',
  'user': u'Usu\xe1rioPar',
  'version': 9},
 {'highway': 'unclassified',
  'id': '23389997',
  'name': None,
  'surface': 'unpaved',
  'timestamp': '2015-04-08T10:59:17Z',
  'uid': '2082896',
  'user': 'zeninguem',
  'version': 3},
 {'highway': 'primary',
  'id': '23389998',
  'name': u'Rua da Esperan\xe7a',
  'surface': 'paved',
  'timestamp': '2015-12-16T12:27:30Z',
  'uid': '31385',
  'user': 'Skippern',
  'version': 9},
 {'highway': 'residential',
  'id': '23389999',
  'lit': 'yes',
  'name': None,
  'timestamp': '2016-09-03T17:56:59Z',
  'uid': '367813',
  'user': u'Usu\xe1rioPar',
  'version': 3},
 {'highway': 'residential',
  'id': '23390001',
  'lit': 'yes',
  'name': None,
  'timestamp': '2016-09-03T17:56:59Z',
  'uid': '367813',
  'user': u'Usu\xe1rioPar',
  'version': 3},
 {'highway': 'pedestrian',
  'id': '23390004',
  'lit': 'yes',
  'n

#### 3.3.7 Importing the Generated JSON file in mongodb
I need now to put this data I just fixed and generated into my local mongodb which is alrady running. I'll name the database **p3_leonardo**, the collection **osm** and run the command below (if you wnat to run it in your local machine just change the local path to the PE_Coast.osm_processed.json file):
<br>
mongoimport --db p3_leonardo --collection osm --file /Users/leo/Dropbox/udacity_nanodegree/P3/PE_Coast.osm_processed.json

<output>
Leonardos-Air:P3 leo$ mongoimport --db p3_leonardo --collection osm --file /Users/leo/Dropbox/udacity_nanodegree/P3/PE_Coast.osm_processed.json
2016-09-16T23:02:38.940-0300	connected to: localhost
2016-09-16T23:02:40.581-0300	imported 35330 documents

#### 3.3.8 Reusable Functions
The functions below will be reused later on when I use some mongodb aggregate functions to analyse the data.

In [62]:
def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db


def run_aggregation(pipeline):
    db = get_db('p3_leonardo')
    result = db.osm.aggregate(pipeline)
    for r in result:
        pprint.pprint(r)

#### 3.3.9 Find the top 10 contributors
TrvrHldr was the top contrinutor to the streets in this area

In [73]:
# find the top 10 contributors to this area
pipeline = [
            {"$group" : {"_id" : "$user", "count" : {"$sum" : 1}}},
            {"$sort" : {"count" : -1}},
            {"$limit" : 10}
            ]

run_aggregation(pipeline)

{u'_id': u'TrvrHldr', u'count': 5552}
{u'_id': u'patodiez', u'count': 5415}
{u'_id': u'Usu\xe1rioPar', u'count': 4356}
{u'_id': u'erickdeoliveiraleal', u'count': 2943}
{u'_id': u'plguedes', u'count': 2048}
{u'_id': u'raphaelmirc', u'count': 1938}
{u'_id': u'elias lopes', u'count': 1881}
{u'_id': u'dbusse', u'count': 1153}
{u'_id': u'Skippern', u'count': 1097}
{u'_id': u'maiafelipe', u'count': 957}


#### 3.3.10 Find the percentage of each street type
We can see here that residential counts for ~72% of all streets while unclassified is only ~7%

In [65]:
# find the percentage of each street type
db = get_db('p3_leonardo')
total_street_count = db.osm.count()
pipeline = [
            {"$group" : {"_id" : "$highway", "count" : {"$sum" : 1}}},
            {"$sort" : {"count" : -1}},
            {"$project" : {"count" : 1, "percentage" : {"$multiply" : [{"$divide" : [100, total_street_count]}, "$count"]}}}
            ]

run_aggregation(pipeline)

{u'_id': u'residential', u'count': 25564, u'percentage': 72.35776960090574}
{u'_id': u'unclassified', u'count': 2371, u'percentage': 6.711010472686102}
{u'_id': u'secondary', u'count': 1111, u'percentage': 3.144636286442117}
{u'_id': u'tertiary', u'count': 1034, u'percentage': 2.9266911972827625}
{u'_id': u'primary', u'count': 965, u'percentage': 2.7313897537503538}
{u'_id': u'service', u'count': 798, u'percentage': 2.258703651287857}
{u'_id': u'footway', u'count': 766, u'percentage': 2.1681290687800736}
{u'_id': u'pedestrian', u'count': 414, u'percentage': 1.1718086611944523}
{u'_id': u'motorway', u'count': 381, u'percentage': 1.0784036229833003}
{u'_id': u'track', u'count': 364, u'percentage': 1.03028587602604}
{u'_id': u'primary_link', u'count': 321, u'percentage': 0.9085762807812057}
{u'_id': u'motorway_link', u'count': 254, u'percentage': 0.7189357486555334}
{u'_id': u'secondary_link', u'count': 226, u'percentage': 0.6396829889612227}
{u'_id': u'trunk', u'count': 175, u'percentage

#### 3.3.11 Find the top contributors to the streets in the db in the last 30 days
UsuárioPar was the top contributor in the last 30 days

In [66]:
# find the top contributors to the streets in the db in the last 30 days

from datetime import datetime
from datetime import timedelta

# first convert the timestamp field from string to datetime in the db
all_streets = db.osm.find()
for street in all_streets:
#     pprint.pprint(street)
    try:
        date_object = datetime.strptime(street["timestamp"], '%Y-%m-%dT%H:%M:%SZ')
        db.osm.update_one({"_id" : street["_id"]}, {"$set" : {"timestamp" : date_object}})
    except TypeError:
        # db already updated
        pass

thirty_days_ago = datetime.now() + timedelta(days=-30)
        
pipeline = [
            {"$match" : {"timestamp": {"$gte": thirty_days_ago}}},
            {"$group" : {"_id" : "$user", "count" : {"$sum" : 1}}},
            {"$sort" : {"count" : -1}}
            ]

run_aggregation(pipeline)

{u'_id': u'Usu\xe1rioPar', u'count': 1553}
{u'_id': u'Alexandre Rui Barboza Lima', u'count': 88}
{u'_id': u'ThiagoPv', u'count': 36}
{u'_id': u'CupBlack', u'count': 11}
{u'_id': u'xamico', u'count': 9}
{u'_id': u'Villenom', u'count': 1}
{u'_id': u'maiafelipe', u'count': 1}


#### 3.3.12 Find the percentage of the streeets that doesn't have lit information
~90% of the entries doesn't have information about whether of or not the streets are lit

In [67]:
# find the percentage of the streeets that doesn't have lit information

pipeline = [
            {"$group" : {"_id" : "$lit", "count" : {"$sum" : 1}}},
            {"$sort" : {"count" : -1}},
            {"$project" : {"count" : 1, "percentage" : {"$multiply" : [{"$divide" : [100, total_street_count]}, "$count"]}}}
            ]

run_aggregation(pipeline)

{u'_id': None, u'count': 31981, u'percentage': 90.52080384941975}
{u'_id': u'yes', u'count': 3235, u'percentage': 9.156524200396262}
{u'_id': u'no', u'count': 114, u'percentage': 0.3226719501839796}


In [68]:
# find the percentage of the streeets that doesn't have surface information

pipeline = [
            {"$group" : {"_id" : "$surface", "count" : {"$sum" : 1}}},
            {"$sort" : {"count" : -1}},
            {"$project" : {"count" : 1, "percentage" : {"$multiply" : [{"$divide" : [100, total_street_count]}, "$count"]}}}
            ]

run_aggregation(pipeline)

{u'_id': None, u'count': 30489, u'percentage': 86.29776393999433}
{u'_id': u'paved', u'count': 2558, u'percentage': 7.240305689215964}
{u'_id': u'unpaved', u'count': 1018, u'percentage': 2.8814039060288703}
{u'_id': u'asphalt', u'count': 914, u'percentage': 2.5870365128785733}
{u'_id': u'paving_stones', u'count': 111, u'percentage': 0.31418058307387486}
{u'_id': u'cobblestone', u'count': 109, u'percentage': 0.3085196716671384}
{u'_id': u'dirt', u'count': 52, u'percentage': 0.1471836965751486}
{u'_id': u'sand', u'count': 33, u'percentage': 0.093405038211152}
{u'_id': u'concrete', u'count': 28, u'percentage': 0.07925275969431078}
{u'_id': u'ground', u'count': 8, u'percentage': 0.022643645626945937}
{u'_id': u'sett', u'count': 2, u'percentage': 0.005660911406736484}
{u'_id': u'earth', u'count': 2, u'percentage': 0.005660911406736484}
{u'_id': u'wood', u'count': 1, u'percentage': 0.002830455703368242}
{u'_id': u'paved;asphalt', u'count': 1, u'percentage': 0.002830455703368242}
{u'_id': u'u

#### 3.3.13 Find the percentage of the streeets that doesn't have oneway information
~60% of the streets doesn't have information about the flow direction

In [69]:
# find the percentage of the streeets that doesn't have oneway information

pipeline = [
            {"$group" : {"_id" : "$oneway", "count" : {"$sum" : 1}}},
            {"$sort" : {"count" : -1}},
            {"$project" : {"count" : 1, "percentage" : {"$multiply" : [{"$divide" : [100, total_street_count]}, "$count"]}}}
            ]

run_aggregation(pipeline)

{u'_id': None, u'count': 21215, u'percentage': 60.04811774695725}
{u'_id': u'no', u'count': 9641, u'percentage': 27.288423436173222}
{u'_id': u'yes', u'count': 4474, u'percentage': 12.663458816869515}


#### 3.3.14 Find the percentage of street without a name
~53% of the streets doesn't have a name in the database

In [71]:
# find the percentage of street without a name
pipeline = [
            {"$match" : {"name": None}},
            {"$group" : {"_id" : "$name", "count" : {"$sum" : 1}}},
            {"$sort" : {"count" : -1}},
            {"$project" : {"count" : 1, "percentage" : {"$multiply" : [{"$divide" : [100, total_street_count]}, "$count"]}}}
            ]

run_aggregation(pipeline)

{u'_id': None, u'count': 18867, u'percentage': 53.40220775544862}


## 4. Conclusion
The streets related data in this dataset has some obvious gaps like ~53% of the streets miss the name tag which makes it not much attractive for a serious use.
### 4.1. Some Thoughts on How to Fill This Gap 
I'll list below some ideas on how we could improve this data.

#### 4.1.1. Brazilian Data Agencies 
Through the wranging process I noticed that some streets has tags like this one.
```xml
<tag k="note:pt" v="Sem nome no IBGE ou no mapa da Prefeitura do Recife em 17/02/2015"/>
```
IBGE is the Brazilian Institute of Geography and Statistics<br>
Prefeitura do Recife is the Recife City Hall<br>

Data from other agencies such as National Departament of Roads, the State Departament of Roads, Cities agencies, National Postal Service and others could be mungled with he data that is already there.

#### 4.1.2. Google Maps
The Google Geolocation API could be used to fill this gap and more. It could be used as well as to confirm some of the existing data such as street names, oneway data and others. This option would come with a cost, though.

#### 4.1.3 Final Notes
I'm sure that the ideas proposed here are just a subset of what can be done to improve this dataset. Note that there's no magic bullet and even data used to fix this dataset can be wrong or damaged in someways. <br>
The OpenStreetMap data biggest strenght is the community and maybe using the power of the community is the way to go. Maybe throwing online competitions and/or creating an app (kind of how Waze started out) to help improving the maps while driving would be a more obvious and general approach to this general and global issue.

http://wiki.openstreetmap.org/wiki/OSM_XML<br>
http://stackoverflow.com/questions/12925052/python-and-default-dict-how-to-pprint<br>
http://stackoverflow.com/questions/613183/sort-a-python-dictionary-by-value?page=1&tab=votes#tab-top<br>
http://stackoverflow.com/questions/16772071/sort-dict-by-value-python<br>
http://stackoverflow.com/questions/2672326/what-does-a-leading-x-mean-in-a-python-string-xaa<br>
http://wiki.openstreetmap.org/wiki/Highways<br>
http://stackoverflow.com/questions/16471332/how-can-i-compare-a-unicode-type-to-a-string-in-python<br>
http://wiki.openstreetmap.org/wiki/Key:lit<br>
http://wiki.openstreetmap.org/wiki/Key:surface<br>
http://stackoverflow.com/questions/31188966/get-percentages-with-mongodb-aggregate-group<br>
http://stackoverflow.com/questions/2900674/how-do-i-convert-a-property-in-mongodb-from-text-to-date-type<br>